In [15]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pickle
from __future__ import print_function, division
from torch.optim import lr_scheduler
from skimage import io, transform
import pandas as pd

batch_size =16 
#PATH  = "/content/model"
  
print(os.getcwd())


/content


학습데이터 경로

In [16]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
class wineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로
            root_dir (string): 모든 이미지가 존재하는 디렉토리 경로
            transform (callable, optional): 샘플에 적용될 Optional transform
        """
        self.wine_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.wine_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

      
        wineFeatures = self.wine_frame.iloc[idx, :-1]
        out = np.zeros((16, 1))
        for index, point in enumerate(wineFeatures):
          out[index] = point
        dataSet = np.dot(out,out.T)
        #끝 4행에 원본데이터 입력
        for i in range(12,16):
          for index, point in enumerate(wineFeatures):
            dataSet[i][index] = point
        #끝 4열에 원본데이터 입력
        for i in range(12,16):
          for index, point in enumerate(wineFeatures):
            dataSet[index][i] = point
        #대각선 끝에 원본데이터 입력 -> 보류
        
        dataSet = np.array([dataSet])
        wineFeatureSet = dataSet.astype('double')
        
        wineQulitySet = self.wine_frame.iloc[idx, -1:]
        sample = {'wineFeatureSet': wineFeatureSet, 'wineQulitySet': wineQulitySet}
        if self.transform:
            sample = self.transform(sample)

        return sample['wineFeatureSet'], int(sample['wineQulitySet'])


class ToTensor(object):
    """numpy array를 tensor(torch)로 변환 시켜줍니다."""

    def __call__(self, sample):
        wineFeatureSet, wineQulitySet = sample['wineFeatureSet'], sample['wineQulitySet']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        result = torch.from_numpy(wineFeatureSet)
        result = result.float()
        return {'wineFeatureSet': result,
                'wineQulitySet': wineQulitySet}

In [18]:
wine_dataset = wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTrainNB.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor(),
                                                                 # transforms.Normalize(mean=(0.5), std=(0.5)),
                                    
                                    ])
                                    )

wine_testSet =wineDataset(csv_file='/content/gdrive/My Drive/wineQualityTestNB.csv',
                                    root_dir='/content/gdrive/My Drive//',
                                    transform=transforms.Compose([ ToTensor()])
                                    )
print (wine_dataset)

image, label = wine_dataset.__getitem__(5)

print(image, label)

image, label = cifar_train.__getitem__(5)

print(image,label)


tensor([[[ 1.0000,  0.0413, -0.4304, -0.7229, -0.9601, -0.7575, -0.9375,
          -0.9447, -0.5901, -0.2248, -0.5169, -0.6232,  1.0000,  1.0000,
           1.0000,  1.0000],
         [ 0.0413,  0.0017, -0.0178, -0.0299, -0.0397, -0.0313, -0.0387,
          -0.0390, -0.0244, -0.0093, -0.0214, -0.0258,  0.0413,  0.0413,
           0.0413,  0.0413],
         [-0.4304, -0.0178,  0.1852,  0.3111,  0.4132,  0.3260,  0.4035,
           0.4066,  0.2540,  0.0968,  0.2224,  0.2682, -0.4304, -0.4304,
          -0.4304, -0.4304],
         [-0.7229, -0.0299,  0.3111,  0.5226,  0.6941,  0.5476,  0.6777,
           0.6829,  0.4266,  0.1625,  0.3736,  0.4505, -0.7229, -0.7229,
          -0.7229, -0.7229],
         [-0.9601, -0.0397,  0.4132,  0.6941,  0.9218,  0.7273,  0.9001,
           0.9070,  0.5666,  0.2158,  0.4962,  0.5983, -0.9601, -0.9601,
          -0.9601, -0.9601],
         [-0.7575, -0.0313,  0.3260,  0.5476,  0.7273,  0.5738,  0.7101,
           0.7156,  0.4470,  0.1703,  0.3915,  0.472

CNN 모델에 csv 데이터를 넣기 위한 트라이

```
# 코드로 형식 지정됨
```



*데이터* 로드, 정규화

In [7]:
cifar_train= dset.CIFAR10("CIFAR10/", train=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),
                          target_transform= None, download=True)
print (cifar_train)
print (len(cifar_train[0][0][0][0]))
print ( len(wine_dataset[0][0][0][0]))
print (len(cifar_train[0][0][0]))
print (len( wine_dataset[0][0][0]))
print (len(cifar_train[0][0]))
print ( len(wine_dataset[0][0]))
print (len(cifar_train[0]))
print ( len(wine_dataset[0]))

cifar_test= dset.CIFAR10("CIFAR10/", train=False, 
                            transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                            ]),target_transform= None, download=True)

Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: CIFAR10/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
32
16
32
16
3
1
2
2
Files already downloaded and verified


 대략적인 데이터 형태

In [0]:
def ComputeAccr(dloader, imodel):
    correct = 0 
    total= 0 
    
    for j, [imgs, labels] in enumerate(dloader):
        img  = Variable(imgs, volatile = True)
        label = Variable(labels)
        
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        #print(label,output_index)
        total += label.size(0)
        correct += (output_index == label).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))
      


데이터 로드 함수

In [0]:
train_loader = torch.utils.data.DataLoader(list(wine_dataset)[:], batch_size= batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(wine_testSet, batch_size= batch_size, shuffle=False, num_workers=2, drop_last=True)

모델 선언

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3, padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0.5),
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*4*4, 64),
            nn.ELU(),
            nn.Dropout2d(0.5),
            nn.BatchNorm1d(64),
            nn.Linear(64,3)
        )
        
        
    def forward(self, x):
        out= self.layer(x)
        out= out.view(batch_size,-1)
        out= self.fc_layer(out)
        return out
    
model= CNN()

loss, optimizer

In [0]:
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

# 불러오기

In [0]:
num_epoch_f=150
#num_epoch_m=100
#num_epoch_l=100

learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)




epoch = 0
#checkpoint = torch.load("/content/gdrive/My Drive/model_entire_batch64_linear.pkl")
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']

#model.eval()
#TotalComputeAccr(test_loader, model)

학습

In [28]:

for i in range(epoch+1 , num_epoch_f):
    print ("%d epoch :" %i) 
    model.train()
    for j,[image, label] in enumerate(train_loader):
        x= Variable(image)
        y_= Variable(label)
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output, y_)
        loss.backward()
        optimizer.step()
        if j%1000 == 0:
            print(j,loss)
            # Save the model
            torch.save({
                'epoch': i,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
          
            }, '/content/gdrive/My Drive/wineQulityMB64.pkl')

    model.eval()
    ComputeAccr(test_loader, model)

1 epoch :
0 tensor(1.2017, grad_fn=<NllLossBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 68.28704071044922
2 epoch :
0 tensor(0.8489, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 69.75308990478516
3 epoch :
0 tensor(0.8098, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 69.6759262084961
4 epoch :
0 tensor(0.7285, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 69.59876251220703
5 epoch :
0 tensor(0.7938, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 70.4475326538086
6 epoch :
0 tensor(0.4375, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 70.4475326538086
7 epoch :
0 tensor(0.4948, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 69.3672866821289
8 epoch :
0 tensor(0.4725, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 69.90740966796875
9 epoch :
0 tensor(0.4928, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 71.14197540283203
10 epoch :
0 tensor(0.5381, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 72.60802459716797
11 epoch :
0 tensor(0.5545, grad_fn=<NllLossBackward>)
Accuracy of Test Data: 70.75617218017578
12 epoch :
